In [ ]:
import fasttext

In [ ]:
# general vars

_dataDir = "../../data/tarea3/"
ftlabel = "__label__"
wVectors = "../../word_vectors/wiki.es.vec"
dimensions = 300

In [ ]:
# utility function for creating input files for fasttext

import string

def FTInputFile(modosFile,xFile,yFile,labelString,outFile):
    modos = []
    i = 0
    with open(modosFile) as f:
        for line in f:
            modos.append(str(i))
            i = i+1

    with open(xFile) as xf, open(yFile) as yf, open(outFile,'w') as out:
        for x,y in zip(xf,yf):
            strX = "".join([c for c in x[:-1] if c not in string.punctuation]).lower()
            out.write(labelString + modos[int(y)] + " " + strX)
            out.write("\n")

# create input files for fasttext

_tmpTrainFile = "_train_"
_tmpDevFile = "_dev_"
_tmpTestFile = "_test_"

In [ ]:
for tt in ["_train_", "_dev_", "_test_"]:
    FTInputFile(_dataDir + "modos.txt",
                    _dataDir + "x" + tt + "modo.txt",
                    _dataDir + "y" + tt + "modo.txt",
                    ftlabel,
                    tt + ".tmp")

In [ ]:
#********************
#*** FINISH 
#*** best recall: 0.8089285714285714 with parameters (300, 0.06, 5, 5)
#*** model stored at _model_ptvec_best.bin
#********************


# grid search for parameters

import sys
import itertools
import os
import shutil


# prefix for the generated model
model_name = "_model_ptvec_"

###
###

if os.path.isfile(model_name + "best.bin"):
    print("Previous existent model")
    best_prev_classifier = fasttext.load_model(model_name + "best.bin")
    best = best_prev_classifier.test(_tmpDevFile + ".tmp", k=1).recall
    print("Best previous recall " + str(round(best,4)))
else:
    print("No previous model")
    best = 0.0

# parameters
N = 4
dim = 300
lrs = [0.05, 0.06, 0.065]
window_sizes = [5]
epochs = [5]


# Perfomr a grid search for all combinations of parameters. 
# For every combination do N repetitions

for (lr,ws,e) in itertools.product(lrs,window_sizes,epochs):
    for i in range(0,N):
        if i == 0:
            print("Checking the combination " + str((lr,ws,e)))
        sys.stdout.write(str(i)+" ")
        classifier = fasttext.supervised(_tmpTrainFile + ".tmp",
                                         model_name,
                                         label_prefix = ftlabel,
                                         dim = dim,
                                         lr = lr,
                                         ws = ws,
                                         epoch = e,
                                         pretrained_vectors = wVectors
                                        )

        result = classifier.test(_tmpDevFile + ".tmp", k=1)

        if result.recall > best:
            best = result.recall
            best_classifier = classifier
            best_parameters = (dim,lr,ws,e)
            shutil.copyfile(model_name + ".bin", model_name + "best.bin")
            print("* updating best recall with parameters " + str(best_parameters) + 
                  " best recall so far: " + str(best))

        os.remove(model_name + ".bin")
        if i == N-1:
            print(" (done)")

print("********************")
print("*** FINISH ")
print("*** best recall: " + str(best) + " with parameters " + str(best_parameters))
print("*** model stored at " + model_name + "best.bin")
print("********************")


In [ ]:
# utility function to select the first prediction from a list of predictions and generate a 1D list of single predictions
def first_prediction(lists_of_predictions):
    out = []
    for predictions in lists_of_predictions:
        out.append(predictions[0])
    return out

In [ ]:
#********************
#*** FINISH 
#*** best f1-macro: 0.629899594733 with parameters (300, 0.06, 5, 5)
#*** model stored at _model_f1_ptvec_best.bin
#********************


# grid search for parameters metric = f1-macro
# change best if there are already better precomputed models for f1-macro
best = 0.62

import sys
import itertools
import os
import shutil
from sklearn import metrics

# prefix for the generated model
model_name = "_model_f1_ptvec_"


############# primero carga los datos para testear el modelo segun f1

import string

def read_text_file_for_ft_input(filename):
    with open(filename) as f:
        out = []
        for line in f:
            # keep tab to separate original concepts from justifications
            strdata = "".join([c for c in line[:-1] if c not in string.punctuation or c == '\t']).lower()
            if strdata == '':
                strdata = ' '
            out.append(strdata)
    return out

def read_numbers_file_for_ft_input(filename):
    with open(filename) as f:
        out = []
        for line in f:
            out.append(int(line))
    return out

dev_x = {}
dev_y = {}

dev_x = read_text_file_for_ft_input(
        _dataDir + "x_dev_modo.txt")
dev_y = read_numbers_file_for_ft_input(
        _dataDir + "y_dev_modo.txt")

modos = []
# load categories first
modosFile = _dataDir + "modos.txt"
with open(modosFile) as f:
    for line in f:
        modos.append(line[:-1])    

#############################


# parameters
N = 4
dim = 300
lrs = [0.05, 0.06, 0.065]
window_sizes = [5]
epochs = [5]


# Perfomr a grid search for all combinations of parameters. 
# For every combination do N repetitions

for (lr,ws,e) in itertools.product(lrs,window_sizes,epochs):
    for i in range(0,N):
        if i == 0:
            print("Checking the combination " + str((lr,ws,e)))
        sys.stdout.write(str(i)+" ")
        classifier = fasttext.supervised(_tmpTrainFile + ".tmp",
                                         model_name,
                                         label_prefix = ftlabel,
                                         dim = dim,
                                         lr = lr,
                                         ws = ws,
                                         epoch = e,
                                         pretrained_vectors = wVectors
                                        )
        
        ## ve que tan bien está el modelo respecto de f1
        predictions_dev_with_labels = classifier.predict(dev_x,k = 3)
        predictions_dev = []
        for pred_list in predictions_dev_with_labels:
            predictions_dev.append([int(p) for p in pred_list])
        f1 = metrics.f1_score(dev_y,first_prediction(predictions_dev),average='macro')

        if f1 > best:
            best = f1
            best_classifier = classifier
            best_parameters = (dim,lr,ws,e)
            shutil.copyfile(model_name + ".bin", model_name + "best.bin")
            print("* updating best f1-macro with parameters " + str(best_parameters) + 
                  " best f1-macro so far: " + str(best))

        os.remove(model_name + ".bin")
        if i == N-1:
            print(" (done)")

print("********************")
print("*** FINISH ")
print("*** best f1-macro: " + str(best) + " with parameters " + str(best_parameters))
print("*** model stored at " + model_name + "best.bin")
print("********************")


In [ ]:
# BIGRAMS

#********************
#*** FINISH 
#*** best f1-macro: 0.646875285904 with parameters (300, 0.1, 5, 5)
#*** model stored at _model_f1_ptvec_ng2_best.bin
#********************

# grid search for parameters metric = f1-macro
# change best if there are already better precomputed models for f1-macro
best = 0.62

import sys
import itertools
import os
import shutil
from sklearn import metrics

word_ngrams = 2
# prefix for the generated model
model_name = "_model_f1_ptvec_ng" + str(word_ngrams) + "_"


############# primero carga los datos para testear el modelo segun f1

import string

def read_text_file_for_ft_input(filename):
    with open(filename) as f:
        out = []
        for line in f:
            # keep tab to separate original concepts from justifications
            strdata = "".join([c for c in line[:-1] if c not in string.punctuation or c == '\t']).lower()
            if strdata == '':
                strdata = ' '
            out.append(strdata)
    return out

def read_numbers_file_for_ft_input(filename):
    with open(filename) as f:
        out = []
        for line in f:
            out.append(int(line))
    return out

dev_x = {}
dev_y = {}

dev_x = read_text_file_for_ft_input(
        _dataDir + "x_dev_modo.txt")
dev_y = read_numbers_file_for_ft_input(
        _dataDir + "y_dev_modo.txt")

modos = []
# load categories first
modosFile = _dataDir + "modos.txt"
with open(modosFile) as f:
    for line in f:
        modos.append(line[:-1])    

#############################


# parameters
N = 3
dim = 300
lrs = [0.1,0.25]
window_sizes = [5]
epochs = [5]


# Perfomr a grid search for all combinations of parameters. 
# For every combination do N repetitions

for (lr,ws,e) in itertools.product(lrs,window_sizes,epochs):
    for i in range(0,N):
        if i == 0:
            print("Checking the combination " + str((lr,ws,e)))
        sys.stdout.write(str(i)+" ")
        classifier = fasttext.supervised(_tmpTrainFile + ".tmp",
                                         model_name,
                                         label_prefix = ftlabel,
                                         dim = dim,
                                         lr = lr,
                                         ws = ws,
                                         epoch = e,
                                         pretrained_vectors = wVectors,
                                         word_ngrams = word_ngrams,
                                         bucket = 2000000
                                        )
        
        ## ve que tan bien está el modelo respecto de f1
        predictions_dev_with_labels = classifier.predict(dev_x,k = 3)
        predictions_dev = []
        for pred_list in predictions_dev_with_labels:
            predictions_dev.append([int(p) for p in pred_list])
        f1 = metrics.f1_score(dev_y,first_prediction(predictions_dev),average='macro')

        if f1 > best:
            best = f1
            best_classifier = classifier
            best_parameters = (dim,lr,ws,e)
            shutil.copyfile(model_name + ".bin", model_name + "best.bin")
            print("* updating best f1-macro with parameters " + str(best_parameters) + 
                  " best f1-macro so far: " + str(best))

        os.remove(model_name + ".bin")
        if i == N-1:
            print(" (done)")

print("********************")
print("*** FINISH ")
print("*** best f1-macro: " + str(best) + " with parameters " + str(best_parameters))
print("*** model stored at " + model_name + "best.bin")
print("********************")
